In [54]:
import arxiv
import fitz  # PyMuPDF
import re
import random
import pandas as pd
import os
import arxiv_tool
import json
import numpy as np
import copy
separator = "\n---SEPARATOR---\n"

# Dataset creation
Here the dataset for the classification (task2) was created and store into the folder task2/dataset 

To create the data the first paper is LLM-based NLG Evaluation: Current Status and Challenges with the number 2402.01383v1
First I download it from arxiv into a pdf.
Then is get the text of the paper and collect its references. The references are now the correct ones. Then some unrelated references need to be added. Then the llm has to identify the correct ones.

In [55]:
408.02304
#paper_id = "2402.01383v1"
#paper_id = "2402.06196"
paper_id = "2408.02304"
#paper_id = "2408.02464"
#paper_id = "2408.02085"
#paper_id = "2311.13731"
#paper_id = "2311.12785"
#paper_id = "2409.15816"
#paper_id = "2409.15180"
#paper_id = "2409.09957" 


In [56]:
paper = next(arxiv.Client().results(arxiv.Search(id_list=[paper_id])))

path = paper.download_pdf()

In [57]:
print(paper.title)
print(paper.summary)
raw = paper._raw
print(raw)
#paper.updated
print(paper.authors)
print(paper.published)

Embedding Compression in Recommender Systems: A Survey
To alleviate the problem of information explosion, recommender systems are
widely deployed to provide personalized information filtering services.
Usually, embedding tables are employed in recommender systems to transform
high-dimensional sparse one-hot vectors into dense real-valued embeddings.
However, the embedding tables are huge and account for most of the parameters
in industrial-scale recommender systems. In order to reduce memory costs and
improve efficiency, various approaches are proposed to compress the embedding
tables. In this survey, we provide a comprehensive review of embedding
compression approaches in recommender systems. We first introduce deep learning
recommendation models and the basic concept of embedding compression in
recommender systems. Subsequently, we systematically organize existing
approaches into three categories, namely low-precision, mixed-dimension, and
weight-sharing, respectively. Lastly, we sum

In [58]:
full_text = arxiv_tool.get_full_text(path)
numbers = arxiv_tool.extract_arxiv_ids(full_text)

In [59]:
references_start = re.search(r'References|Bibliography', full_text, re.IGNORECASE)
references_text = full_text[references_start.start():]
numbers = arxiv_tool.extract_arxiv_ids(references_text)
#Remove the id of the original paper from its references
if paper_id in numbers:
    numbers.remove(paper_id)
print(len(numbers))

7


In [60]:
print(len(numbers))
df_all = pd.DataFrame({'arxiv_id': numbers})
 # Save the DataFrame to a CSV file
#df_all.to_csv('task2/dataset/'+paper_id+'_all.csv', index=False)

7


In [61]:
def create_json(ids):
    if len(ids)>50:
        ids = ids[:50]
    json_data = {}
    for id in ids:
        paper = next(arxiv.Client().results(arxiv.Search(id_list=[id])))
        json_data[id] = {'title':paper.title,'abstract':paper.summary,'date':paper.published.isoformat(),'label':1}  
    return json_data

data  = create_json(numbers)
#data
print(f"There have been {len(data.keys())} reference documents extracted from the original paper.")

There have been 7 reference documents extracted from the original paper.


In [49]:
#print(data['2305.10403']['date'])
#print(data['2308.07201']['date'])
#data['2305.10403']['date']<data['2308.07201']['date']
print(data)
print(paper_id)

{'2202.01279': {'title': 'PromptSource: An Integrated Development Environment and Repository for Natural Language Prompts', 'abstract': 'PromptSource is a system for creating, sharing, and using natural language\nprompts. Prompts are functions that map an example from a dataset to a natural\nlanguage input and target output. Using prompts to train and query language\nmodels is an emerging area in NLP that requires new tools that let users\ndevelop and refine these prompts collaboratively. PromptSource addresses the\nemergent challenges in this new setting with (1) a templating language for\ndefining data-linked prompts, (2) an interface that lets users quickly iterate\non prompt development by observing outputs of their prompts on many examples,\nand (3) a community-driven set of guidelines for contributing new prompts to a\ncommon pool. Over 2,000 prompts for roughly 170 datasets are already available\nin PromptSource. PromptSource is available at\nhttps://github.com/bigscience-worksh

In [50]:
# These data will also be helpful in task3. Lets store them in task3/dataset
def write_to_json(json_data,file_name):
    with open(file_name+'.json', 'w') as file:
        json.dump(json_data, file, indent=4)
write_to_json(data,'task3/dataset/'+paper_id+'ref')

In [28]:
#Lets not forget that the original paper should also be stored
original_paper = {'id':paper_id,'title':paper.title,'abstract':paper.summary,'date':paper.published.isoformat(),'fulltext':full_text,'ref':numbers}
print(original_paper['title'])
write_to_json(original_paper,'task3/dataset/'+paper_id+'data')

Unleashing the Power of Data Tsunami: A Comprehensive Survey on Data Assessment and Selection for Instruction Tuning of Language Models


In [29]:
def add_unrelated(related_data,cut_date,query,num):
    all_data = {}
    client = arxiv.Client()
    search = arxiv.Search(
    query = query,
    max_results = num+50,
    # sort_by=arxiv.SortCriterion.SubmittedDate
    # sort_by = arxiv.SortCriterion.Relevance #Is default

    )

    results = list(client.results(search))
    random.shuffle(results)
    # Lists to hold all IDs and their labels
    all_ids = []
    i = 0
    while len(all_ids)<num:
        result = results[i]
        arxiv_number = result.entry_id.split('/')[-1]
        arxiv_number = arxiv_number.split('v')[0]
        submission_date = result.published.isoformat()
        #check if already in list of sources (hence relevant) and before the overview
        if (arxiv_number not in related_data.keys()) and (submission_date<cut_date):
            all_ids.append(arxiv_number)
            all_data[arxiv_number] = {'title':result.title,'abstract':result.summary,'date':result.published.isoformat(),'label':0}  #No match/ label  0
        #print(submission_date)
        print(result.title)
        i+=1
    print(related_data.keys())
    selected_numbers = random.sample(list(related_data.keys()), num)

    # Add matching IDs to the list
    for matching_id in selected_numbers:
        all_data[matching_id] = related_data[matching_id]


    keys = list(all_data.keys())
    random.shuffle(keys)

    # Create a new dictionary with shuffled keys
    shuffled_dict = {key: all_data[key] for key in keys}
    return shuffled_dict

sample_data = add_unrelated(data,original_paper['date'],'computer science',min(10,len(data.keys())))
#sample_data


Abstract Stobjs and Their Application to ISA Modeling
Computer Science
Saying Hello World with MOLA - A Solution to the TTC 2011 Instructive Case
Simulating Grover's Quantum Search in a Classical Computer
Sequentiality vs. Concurrency in Games and Logic
Decidability of higher-order matching
Crypto Makes AI Evolve
Typed Operational Semantics for Dependent Record Types
Advances in Artificial Intelligence Require Progress Across all of Computer Science
Artificiality in Social Sciences
dict_keys(['2303.09540', '2303.08774', '2402.16827', '2401.13229', '2405.20541', '2407.14985', '2406.09334', '2402.17327', '2405.12186', '2309.16609', '2203.14544', '1512.02985', '2006.14651', '2401.06692', '2312.06254', '2310.13032', '2311.14736', '2403.16898', '1702.05962', '2307.06290', '2308.07201', '2105.10446', '2305.09246', '2403.12776', '2307.08701', '2406.14491', '2405.20456', '2311.09783', '2109.06379', '1810.04805', '2305.14233', '2002.06305', '2406.13542', '2304.06767', '2311.15653', '1903.09722'

In [47]:
random.sample(list(data.keys()), 2)


['2405.16771', '2403.01121']

In [48]:
write_to_json(sample_data,'task2/dataset/'+paper_id)

In [49]:
sample_data = add_unrelated(data,original_paper['date'],'computer science',min(len(data.keys()),100))
write_to_json(sample_data,'task2/dataset/'+paper_id+'all')

FAIR and Open Computer Science Research Software
Computer Science for Future -- Sustainability and Climate Protection in the Computer Science Courses of the HAW Hamburg
Data science and Machine learning in the Clouds: A Perspective for the Future
Computer simulations in science and engineering - Concepts - Practices - Perspectives
Why The Trans Programmer?
Generative AI has lowered the barriers to computational social sciences
A Polynomial Translation of pi-calculus FCPs to Safe Petri Nets
A Cognitive Science perspective for learning how to design meaningful user experiences and human-centered technology
Computational Skills by Stealth in Secondary School Data Science
The Difficulties of Addressing Interdisciplinary Challenges at the Foundations of Data Science
Saying Hello World with MOLA - A Solution to the TTC 2011 Instructive Case
Proceedings of the 7th European Conference on Python in Science (EuroSciPy 2014)
Materials science and engineering: New vision in the era of artificial i

### Dataset with full text

In [62]:
def get_full_text(id):
    #Find paper in arxiv
    paper = next(arxiv.Client().results(arxiv.Search(id_list=[id])))
    # Download the PDF
    directory_path = './task3/'+paper_id  
    os.makedirs(directory_path, exist_ok=True)
    pdf_path = paper.download_pdf(dirpath=directory_path)
    # Open the downloaded PDF
    document = fitz.open(pdf_path)

    # Extract text from each page
    text = ""
    for page_num in range(len(document)):
        page = document.load_page(page_num)
        text += page.get_text()
    return text

In [63]:
#Attention: takes long and a lot of space, pdfs get stored can be used for chroma though.
def create_dict_full_text(numbers,file_name):
    json_data = {}
    for number in numbers:
        text = get_full_text(number)
        json_data[number] = text
        print(number)
    with open(file_name+'.json', 'w') as file:
        json.dump(json_data, file, indent=4)
    return json_data
    
data = create_dict_full_text(numbers,'./task3/dataset/'+paper_id+'full_texts')

2112.01944
2006.04466
2103.06124
2006.05623
1911.02079
2010.11305
2006.14827


In [19]:
data

{'2112.01944': 'arXiv:2112.01944v1  [cs.IR]  3 Dec 2021\nTowards Low-loss 1-bit Qantization of User-item\nRepresentations for Top-K Recommendation\nYankai Chen1, Yifei Zhang1, Yingxue Zhang2, Huifeng Guo3,\nJingjie Li3, Ruiming Tang3, Xiuqiang He3, Irwin King1\n1The Chinese University of Hong Kong, 2Huawei Noah’s Ark Lab Canada, 3Huawei Noah’s Ark Lab\n{ykchen,yfzhang,king}@cse.cuhk.edu.hk,{yingxue.zhang,huifeng.guo,lijingjie1,tangruiming,hexiuqiang1}@huawei.com\nAbstract\nDue to the promising advantages in space compression and in-\nference acceleration, quantized representation learning for recom-\nmender systems has become an emerging research direction re-\ncently. As the target is to embed latent features in the discrete em-\nbedding space, developing quantization for user-item representa-\ntions with a few low-precision integers confronts the challenge of\nhigh information loss, thus leading to unsatisfactory performance\nin Top-K recommendation. In this work, we study the proble

In [53]:
paper_id

'2311.12785'